# Footprint Simulator

In [75]:
import tkinter

from pathlib import Path
from tkinter import ttk

import floor_plan

class FootprintSimulator:
    N, W, E, S = tkinter.N, tkinter.W, tkinter.E, tkinter.S

    def __init__(self, root, path):
        """
        Parameters
        ----------
        root : tkinter.Tk
        path : pathlib.Path
            A data folder path that includes layout folders.
        
        Returns
        -------
        None
        """
        
        self.root = root
        self.path = path
        
        self.root.title('Footprint Simulator')
        
        # widgets
        self.mainframe = ttk.Frame(root, padding = (3,3,12,12))

        self._time_l = ttk.Label(self.mainframe, text = "Time")  # l means label
        self._time = ttk.Label(self.mainframe, text = "Time")
        
        self._activity_l = ttk.Label(self.mainframe, text = "Activity")
        self._activity = ttk.Label(self.mainframe, text = "Activity")
        
        # self._figure = ttk.Frame(self.mainframe, borderwidth = 5, relief = "ridge", width = 200, height = 100)
        self._figure = tkinter.Canvas(self.mainframe, borderwidth = 5, relief = 'ridge', bg = 'white')
        
        self.layouts = [p for p in self.path.iterdir()]
        self.layouts.remove(path / '.ipynb_checkpoints')
        self.layouts_name = [p.name for p in self.layouts]
        self._layouts_name = tkinter.StringVar(value = self.layouts_name)
        self._layout_l = ttk.Label(self.mainframe, text = "Layout")
        self._layout = tkinter.Listbox(self.mainframe, listvariable = self._layouts_name, height = 3)
        self._layout.bind('<<ListboxSelect>>', self.show_data_list)
        
        self.data = []
        self.data_name = []
        self._data_name = tkinter.StringVar(value = self.data_name)
        self._data_l = ttk.Label(self.mainframe, text = "Data")
        self._data = tkinter.Listbox(self.mainframe, listvariable = self._data_name, height = 3)
        self._data.bind('<<ListboxSelect>>', self.draw_figure)
        
        self._simulation_l = ttk.Label(self.mainframe, text = "Simulation")
        self._simulation = ttk.Button(self.mainframe, text="Start")

        # grid geometry manager
        self.mainframe.grid(row = 0, column = 0, sticky = (self.N, self.S, self.E, self.W)) 
        self._time_l.grid(row = 0, column = 0)
        self._time.grid(row = 0, column = 1)
        self._activity_l.grid(row = 0, column = 2)
        self._activity.grid(row = 0, column = 3) 
        self._figure.grid(row = 1, column = 0, columnspan = 6, sticky = (self.N, self.S, self.E, self.W)) 
        self._layout_l.grid(row = 2, column = 0)
        self._layout.grid(row = 2, column = 1)
        self._data_l.grid(row = 2, column = 2)
        self._data.grid(row = 2, column = 3)
        self._simulation_l.grid(row = 2, column = 4)
        self._simulation.grid(row = 2, column = 5) 
        
        # resizable
        self.root.columnconfigure(0, weight = 1)
        self.root.rowconfigure(0, weight = 1)
        self.mainframe.columnconfigure(0, weight = 1)
        self.mainframe.columnconfigure(1, weight = 1)
        self.mainframe.columnconfigure(2, weight = 1)
        self.mainframe.columnconfigure(3, weight = 1)
        self.mainframe.rowconfigure(1, weight = 1)
        
    def show_data_list(self, *args):
        indexes = self._layout.curselection()
        index = int(indexes[0])
        layout_path = self.layouts[index]
        self.data = []
        for p in layout_path.iterdir():
            if p.is_dir() and p.name != '.ipynb_checkpoints':
                self.data.append(p)
        self.data_name = [p.name for p in self.data]
        self._data_name.set(self.data_name)
            
        
    def draw_figure(self, *args):
        indexes = self._data.curselection()
        index = int(indexes[0])
        data_path = self.data[index]
        
        fp = floor_plan.FloorPlan()
        fp.load_layout(layout_path)



root = tkinter.Tk()
path = Path('./layout_data')
FootprintSimulator(root, path)
root.mainloop()

In [ ]:
    def draw_figure(self, *args):
        indexes = self._data.curselection()
        index = int(indexes[0])
        data_path = self.data[index]
        
        fp = floor_plan.FloorPlan()
        fp.load_layout(layout_path)
        
        house_js = Load.load_se_map(path_l[0], path_l[1], path_l[2])
        rooms, T_Bs, furnitures, doors, walls, self.lims = Load.dic2house(house_js)
        Fur_set, self.scale, self.IsROT, Fur_indexs = SP.layoutplot(Canvas, [640, 370], rooms, T_Bs, furnitures, doors,
                                                                    walls, self.lims, showbg=False)
        
    def layoutplot(Canvas, Can_size, Rooms, T_Bs, Furns, Doors, Walls, Lims, showbg=True):
        Colors = {'Bedroom': 'blue', 'Kitchen': 'green', 'Livingroom': 'red', 'Toilet': 'yellow', 'Bathroom': 'purple'}
        NameFur_set = ['Bed', 'Wardrobe', 'Desk', 'Kitchen_Stove', 'Cupboard', 'Refrigerator', 'Wash_Machine',
                       'Trash_Bin', 'Dinner_Table', 'Sofa']
        Fur_set, Text = [[] for i in range(10)], [] # text is for locating the index of all text objective in canvas
        [Can_W, Can_H] = Can_size

        def scale(W, H):
            scales = [2, 2.5, 5]
            for i in range(3):
                if W/scales[i]<Can_W and H/scales[i]<Can_H: return scales[i]

        T_W, T_H = Lims[0][1]-Lims[0][0], Lims[1][1]-Lims[1][0]
        Offset = [(Lims[0][1]+Lims[0][0])//2, (Lims[1][1]+Lims[1][0])//2] #the real coordinates of the center of house
        Rotate_Flag = True if T_H>T_W else False
        Scale = scale(T_H, T_W) if Rotate_Flag else scale(T_W, T_H)
        font = 'Times 12'
        if Scale>2: font = 'Times 10' if Scale==2.5 else 'Times 7'
        T_rooms = Rooms + T_Bs
        for room in T_rooms:
            bcolor = Colors[room[-1]] if showbg else 'white'
            canxys = realxy2can(room[:4], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag) # [x0, y0, x1, y1]
            Canvas.create_rectangle(canxys[0], canxys[1], canxys[2], canxys[3], fill=bcolor, outline='')
            if room[-1] == 'Toilet' or room[-1] == 'Bathroom':
                Canvas.create_line(canxys[0], canxys[1], canxys[0], canxys[3], width=3)
                Canvas.create_line(canxys[0], canxys[1], canxys[2], canxys[1], width=3)
                Canvas.create_line(canxys[2], canxys[3], canxys[2], canxys[1], width=3)
                Canvas.create_line(canxys[2], canxys[3], canxys[0], canxys[3], width=3)
        for i, room in enumerate(Furns):
            Text.append({})
            fcolor = 'white' if showbg else 'black'
            for fur in room:
                name = fur[-1]
                canxys = realxy2can(fur[:4], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag)
                fur_Obj = Canvas.create_rectangle(canxys[0], canxys[1], canxys[2], canxys[3], outline=fcolor)
                text_Obj = Canvas.create_text((canxys[0]+canxys[2])//2, (canxys[1]+canxys[3])//2,
                                              text=Code[name], font=font, fill=fcolor)
                if name in NameFur_set:
                    j = NameFur_set.index(name)
                    Fur_set[j].append(fur_Obj)
                    Fur_set[j].append(text_Obj)
                    Text[i][name] = [text_Obj]
                elif name == 'TV':
                    Fur_set[9].append(fur_Obj)
                    Fur_set[9].append(text_Obj)
                    Text[i][name] = [text_Obj]
                elif name == 'Nightstand':
                    Fur_set[0].append(fur_Obj)
                    Fur_set[0].append(text_Obj)
                    if name not in Text[i]: Text[i][name] = [text_Obj]
                    else: Text[i][name].append(text_Obj)
                else:
                    if name not in Text[i]: Text[i][name] = [text_Obj]
                    else: Text[i][name].append(text_Obj)
                    if i == 0:
                        Fur_set[2].append(fur_Obj)
                        Fur_set[2].append(text_Obj)
                    else:
                        Fur_set[8].append(fur_Obj)
                        Fur_set[8].append(text_Obj)
        for wall in Walls:
            canxy0 = realxy2can(wall[0], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag, Is_Rect=False)
            canxy1 = realxy2can(wall[1], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag, Is_Rect=False)
            Canvas.create_line(canxy0[0], canxy0[1], canxy1[0], canxy1[1], width=3)
        for door in Doors:
            canxys = realxy2can(door[:4], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag)
            Canvas.create_line(canxys[0], canxys[1], canxys[2], canxys[3], fill='white', width=3)
        return Fur_set, Scale, Rotate_Flag, Text